<a href="https://colab.research.google.com/github/RohithJ11/NLP_Privacy_Policies/blob/main/TrainedBARTmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing Required Libraries

In [ ]:
pip install transformers datasets torch


#Loading the Dataset from HuggingFace

In [ ]:
from datasets import load_dataset

dataset = load_dataset("RohithJ/cleaned_dataset_prvy_plcy")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/51 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7 [00:00<?, ? examples/s]

#Tokenizing the data using BERT's tokenizer

In [ ]:
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_function(examples):
    inputs = tokenizer(examples['Content'], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples['Summary_of_Content'], padding="max_length", truncation=True, max_length=128)
    return {
        'input_ids': inputs.input_ids,
        'attention_mask': inputs.attention_mask,
        'decoder_input_ids': outputs.input_ids,
        'decoder_attention_mask': outputs.attention_mask,
        'labels': outputs.input_ids
    }

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

#Configuring the EncoderDecoderModel using BERT for both encoder and decoder.

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased')


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.e

#Installing the accelerate library

In [ ]:
pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
pip install transformers[torch] -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 41.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


#Defining and Training the BERT Model

In [ ]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    save_strategy='steps',
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    load_best_model_at_end=True,
    eval_steps=500
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


TrainOutput(global_step=39, training_loss=4.914579636011368, metrics={'train_runtime': 1314.1393, 'train_samples_per_second': 0.116, 'train_steps_per_second': 0.03, 'total_flos': 93858477373440.0, 'train_loss': 4.914579636011368, 'epoch': 3.0})

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Assuming you have a model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Save the model and tokenizer
model_save_path = '/content/drive/My Drive/Colab Notebooks/my_trained_model'
tokenizer_save_path = '/content/drive/My Drive/Colab Notebooks/my_trained_model'

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('/content/drive/My Drive/Colab Notebooks/my_trained_model/tokenizer_config.json',
 '/content/drive/My Drive/Colab Notebooks/my_trained_model/special_tokens_map.json',
 '/content/drive/My Drive/Colab Notebooks/my_trained_model/vocab.txt',
 '/content/drive/My Drive/Colab Notebooks/my_trained_model/added_tokens.json')

In [1]:
!pip install PyMuPDF
import fitz  # Import PyMuPDF
import re    # Import the regex module

pdf_path = '/content/Privacy_Policy_Pfizer.pdf'
doc = fitz.open(pdf_path)

text = ""
for page in doc:
    page_text = page.get_text()
    # Remove commas and dots from the text
    page_text = page_text.replace(',', '').replace('.', '')
    # Replace multiple spaces with a single space
    page_text = re.sub(r'\s+', ' ', page_text)
    text += page_text

print(text)  # Output the cleaned text to verify


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 25.0 MB/s eta 0:00:00
 Pfizer Privacy Policy US Effective August 23rd 2023 This Privacy Policy describes the information that we collect about you on the website from which you linked (“Site”); how we use that information; how we protect it; and the choices you may make with respect to it Except as specifically provided below in the Additional State Disclosures Section this Privacy Policy does not apply to any other information collected by Pfizer by or through any other means such as information collected offline When we refer to ourselves as “we" or "Pfizer" we mean Pfizer Inc and all of its subsidiary companies Our Site may contain links to third party websites/content/services that are not owned or controlled by Pfizer Pfizer is not responsible for how these properties operate or treat your personal data so we recommend that you read the privacy 

In [ ]:
from transformers import EncoderDecoderModel, BertTokenizer
import torch

# Load the trained model and tokenizer from your specified path
model_path = '/content/drive/My Drive/model_directory'
model = EncoderDecoderModel.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Prepare the input for summarization
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
inputs = inputs.to(device)  # Ensure tensors are on the correct device

# Ensure decoder_start_token_id is set, default to BERT's [CLS] token if not found
decoder_start_token_id = model.config.decoder.bos_token_id if model.config.decoder.bos_token_id is not None else tokenizer.cls_token_id

# Generate summary using the correct decoder_start_token_id
# Generate summary using refined settings
summary_ids = model.generate(
    inputs['input_ids'],
    decoder_start_token_id=decoder_start_token_id,
    num_beams=4,
    max_length=200,
    min_length=50,
    early_stopping=True,
    no_repeat_ngram_size=2,
    repetition_penalty=2.0,
    temperature=0.9,
    top_k=50,
    top_p=0.95
)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Improved Summary:", summary)


Improved Summary: the themonmonon.. ; ; ep epptptopopoopoopapapepeptptpypypipipopropopopsops ps pspspspppopoponponpoppoppapapoloplpapopppopgpopupoppyopurourototoitoitoparapara polyiprupruproproprorococotomtomporporepteptliplipriproptoptopopspapopeopпopititop zipiproppedpedpop p p sp sp sepipcpopaproptripopoopompop lipipypophipipetyoppipiitiponputputop trip trip pip pip post post posts postsop downop opoputop popopumpopampopsopoprosrop prepponپyp pa pa para para poly polyopspastasopscrattatapapeapdpypappicpiciphopoplumrimrimroprupramramongopdropop


In [ ]:
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example of proper tokenization and dataset preparation
def encode(examples):
    inputs = tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=128)
    outputs = tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=128)
    return {
        'input_ids': inputs.input_ids,
        'attention_mask': inputs.attention_mask,
        'decoder_input_ids': outputs.input_ids,
        'labels': outputs.input_ids  # labels are usually the target inputs
    }

train = train.map(encode, batched=True)
validation= validation.map(encode, batched=True)


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',  # Changed from 'steps' to 'epoch'
    save_strategy='steps',
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,  # Added to help with small batch sizes
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,  # Adjust logging to every 100 steps
    save_steps=500,
    load_best_model_at_end=True,
    eval_steps=500,
    prediction_loss_only=True  # Helps focus on the primary metric
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset
)

trainer.train()


In [2]:
pip install PyPDF2 transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

#Summarized Content of Privacy Policy Document

In [5]:
from transformers import pipeline

def summarize_text(text):
    # Load the summarization pipeline
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
    # Initialize variables to store results
    summarized_text = ""
    num_tokens = 1024  # Set token limit based on model capability, adjust as needed

    # Split the text into chunks of appropriate size
    parts = [text[i:i + num_tokens] for i in range(0, len(text), num_tokens)]

    # Summarize each part and combine
    for part in parts:
        summary = summarizer(part, max_length=150, min_length=40, do_sample=False)
        summarized_text += summary[0]['summary_text'] + " "

    return summarized_text

# Example usage
pdf_file_path = '/content/Privacy_Policy_Pfizer.pdf'  # replace with your PDF file path
extracted_text = extract_text_from_pdf(pdf_file_path)
summary = summarize_text(extracted_text)
print(summary)


Your max_length is set to 150, but your input_length is only 126. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=63)


 This Privacy Policy describes the information that we collect about you on the website from which you are linked . When we refer to ourselves as “ we" or "Pfizer ," we mean  we . Pfizer Inc. and all of its subsidiary companies . Our Site may contain links to third party sites/content/services that are not owned or controlled by Pfizer .  Our Privacy Policy is organized in the following sections . For additional information relating to your rights and our processing of your personal information, please visit the Additional State Disclosures section below . e, and retain your  personal information .  We collect informat ion about you directly from you, automatically from you . We collect information about you from your contact with us . We also collect data about you, and use it as a tool to help us understand our privacy .  We collect personal data you choose to provide, e.g., through registrations, applications and surveys, and in connection with your inquiries and purchases . We and 